In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Step 1: Load the real-life Air Passengers dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
data = pd.read_csv(url)
# Extract the 'Passengers' column as our values
values = data['Passengers'].values
print(f"First 5 passenger counts: {values[:5]}")

First 5 passenger counts: [112 118 132 129 121]


In [3]:
# Step 2: Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values.reshape(-1, 1))
print(f"First 5 passenger counts: {scaled_values[:5]}")
print(f"Length is : {scaled_values.shape}")

First 5 passenger counts: [[0.01544402]
 [0.02702703]
 [0.05405405]
 [0.04826255]
 [0.03281853]]
Length is : (144, 1)


In [4]:
# Create sequences: Use past 'seq_length' months to predict the next month's passengers
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]  # Input sequence of past months
        y = data[i + seq_length]     # Target: next month's passenger count
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [5]:
seq_length = 3  # Use 3 months of past data to predict the next
X, y = create_sequences(scaled_values, seq_length)
print(X[:5])
print(y[:5])

[[[0.01544402]
  [0.02702703]
  [0.05405405]]

 [[0.02702703]
  [0.05405405]
  [0.04826255]]

 [[0.05405405]
  [0.04826255]
  [0.03281853]]

 [[0.04826255]
  [0.03281853]
  [0.05984556]]

 [[0.03281853]
  [0.05984556]
  [0.08494208]]]
[[0.04826255]
 [0.03281853]
 [0.05984556]
 [0.08494208]
 [0.08494208]]


In [6]:
# Split into train and test sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [7]:
# Reshape data to [samples, time steps, features] for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape[0])
print(X_train.shape[1])
print(X_train[3])

112
3
[[0.04826255]
 [0.03281853]
 [0.05984556]]


In [8]:
# Step 3: Build the LSTM model
model = keras.Sequential([
    layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),  # LSTM layer with 50 units
    layers.Dense(1)  # Output layer predicting one value (next month's passengers)
])

C:\Users\amang\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Step 4: Compile the model with Mean Absolute Error as a metric
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanAbsoluteError()])

In [10]:
# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test), verbose=0)

In [11]:
# Step 6: Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MSE (Loss): {test_loss:.4f}')
print(f'Test Mean Absolute Error: {test_mae:.4f}')

Test MSE (Loss): 0.0166
Test Mean Absolute Error: 0.1052


In [12]:
# Step 7: Make a prediction for a new sequence
test_input = scaled_values[-seq_length:]  # Use last 3 months from dataset
test_input = test_input.reshape((1, seq_length, 1))  # Reshape for model
prediction = model.predict(test_input, verbose=0)
# Inverse transform to get actual passenger number
predicted_passengers = scaler.inverse_transform(prediction)[0][0]
print(f'Predicted next month passenger count: {predicted_passengers:.2f}')

Predicted next month passenger count: 423.20
